# ДЗ Парсер сайта

http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/

#### Библиотеки

In [1]:
#!pip3 install requests bs4 pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

#### Получение и преобразование html страницы списка квартир и страницы самой квартиры

In [2]:
table_content = requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/')
table_content

<Response [200]>

In [3]:
table_content = BeautifulSoup(table_content.text, 'lxml')
table_content # разобрали страницу на понятный, более менее упорядоченный html код

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="780e99506e03edb5" name="yandex-verification"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Квартиры в Магнитогорске - 3457 объявлений о продаже в Магнитогорске.</title>
<meta content="Квартиры" http-equiv="keywords"/>
<meta content="Объявления от агентств недвижимости и частных лиц о продаже квартир в Магнитогорске. Купите квартиру на рынке вторичного жилья недорого и без посредников " http-equiv="description"/>
<link href="/img/favicon2.png" rel="SHORTCUT ICON"/>
<link href="http://r.citystar.ru/img/_min/realty.adv-list.min.css" rel="stylesheet" type="text/css"/>
<script src="http://r.citystar.ru/js/jquery-1.7.2.min.js?abcd2ba13348f178b17141b445bc99f1917d47af" type="text/javascript"></script>
<script src="http://r.citystar.ru/js/05-jquery-ui-1.8

In [4]:
flat_page = requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23240323.html')
flat_page = BeautifulSoup(flat_page.text, 'lxml')
flat_page #посмотрим на код страницы квартиры, чтобы понять, как нам извлекать данные

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="780e99506e03edb5" name="yandex-verification"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Квартира 44,50 м2 по улице Суворова 138 в Магнитогорске - продается за 3550000 рублей. Объявление от продавца  на CityStar.</title>
<meta content="Квартира" http-equiv="keywords"/>
<meta content="Продается Двухкомнатная квартира по адресу: Магнитогорск, ул. Суворова, 138,  район Правобережный за 3550000 рублей." http-equiv="description"/>
<link href="/img/favicon2.png" rel="SHORTCUT ICON"/>
<link href="http://r.citystar.ru/img/_min/realty.adv-card.min.css" rel="stylesheet" type="text/css"/>
<script src="http://r.citystar.ru/js/jquery-1.7.2.min.js?abcd2ba13348f178b17141b445bc99f1917d47af" type="text/javascript"></script>
<script src="http://r.citystar.ru/js/

### Извлечем параметры квартиры с каждой из страниц

Перед извлечением параметров было проведено ручное исследование страниц квартир и страниц со спискоми квартир. Было выявлено, что информация на странице квартиры наиболее полная, а также хранится в более удобном формате. Следовательно парсить данные по квартирам со страницы со списком квартир имеет смысл только для получения айди объявления. Этот айди затем будет использоваться в качестве индекса в нашей таблице с данными.
<img src="../datasets/flat_page.png" width="500"> <img src="../datasets/table_content.png" width="550">

#### 1) Парсим идентификаторы квартир

In [5]:
# Для начала найдем сколько страниц нам придется парсить
div = table_content.find('div', 'pager__pages')
div

<div class="pager__pages"><span class="pager__page_state_current">1</span><a class="pager__page" href="/realty/prodazha-kvartir/?p=2">2</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=3">3</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=4">4</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=5">5</a><a class="pager__page common-link-visited" href="/realty/prodazha-kvartir/?p=6">...</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=21">21</a></div>

In [6]:
numeric_list = [int(i.get_text()) for i in div.find_all('a') if i.get_text().isdigit()]
print(numeric_list)
page_count = max(numeric_list) # Лучше было сделать так: page_count = int(list(div)[-1].get_text())
page_count # Значение можно было получить взяв последний элемент списка, но мало ли как расположит автор сайта нумерацию страниц :)

[2, 3, 4, 5, 21]


21

In [7]:
list_pages = range(1, page_count + 1) # Прямо во время выполнения задания страниц стало 21, вместо 20)
list_pages

range(1, 22)

При просмотре кода страницы с объявлениями видно, что айди квартиры находится в первом и во втором столбцах таблицы. Мы будем брать данные из первого столбца.
<img src="../datasets/flat_id.png">

In [8]:
# Находим на странице таблицу, отсеиваем ненужные строки, получаем список из айдишников квартир
table = table_content.find('table', 'table')
table

<table class="table" id="tbl_10043">
<tr class="headings">
<th class="notebook-column" rowspan="2">
<div class="popup-panel">
<a class="trigger notebook-counter" href="javascript:void(0)" onclick="go('/realty/prodazha-kvartir/?notebook=1')"><span id="note_count_10043">
                            0</span> </a>
<div class="popup" style="width: 140px;">
<div style="padding-bottom: 0px;">
<a class="trigger notebook-counter" href="javascript:void(0)" onclick="go('/realty/prodazha-kvartir/?notebook=1')">Показать отмеченные [+]</a></div>
<a class="cs-cancel-url" href="javascript:void(0)" id="clear_notebook_10043" style="display: none; margin-bottom: 5px;" title="">
                                Очистить блокнот [-]</a>
<div>
<a class="cs-cancel-url" href="/realty/prodazha-kvartir/?" id="cancel_notebook_filter_10043" onclick="go('/realty/prodazha-kvartir/?')" style="display: none;">Показать все [x]</a>
</div>
</div>
</div>
</th>
<th class="heading" rowspan="2">
<div class="heading-block">Да

In [9]:
# Нас инетересует содержание столбеца notebook-column
tr_list = [td for td in table.find_all('tr', recursive=False) if td.find('td', 'notebook-column')]
tr_list[0] # 30 штук

<tr>
<td class="notebook-column"><input name="cs_note_tg" type="checkbox" value="23177987"/></td>
<td class="date">10 окт<br/><span class="time">14:21</span></td><td class="image"><div class="image-block"><span class="ibf" id="ibfbut_23177987" title="Добавить в избранное"></span><a href="/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23177987.html"><img alt="" src="http://s3.citystar.ru/bk/0316199efc0797a13217605949fea042-crop-87x65.jpg"/></a></div></td><td class="flat-type"><table style="border:none;"><tr><td style="border:none;"><b>1К</b></td><td style="border:none;">брежневка</td></tr></table></td><td class="address">Ленинский<br/><a href="/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23177987.html">Октябрьская, 32/1</a></td><td class="floor">5<span class="slash">/</span>5</td><td class="square">30,5<span class="slash">/</span>18<span class="slash">/</span>6 <span class="colored-text-1">м2</span></td><td class="price"><span>2 550 000</span><span class="colored-text-1"> р.</

In [10]:
id_list = [i.find('input', {'name': 'cs_note_tg'})['value'] for i in tr_list]
link_list = [i.find_all('a')[0]['href'] for i in tr_list]
list(zip(id_list, link_list)) # тоже 30 штук, осталось поставить все "на поток"

[('23177987',
  '/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23177987.html'),
 ('23177988',
  '/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23177988.html'),
 ('23182085',
  '/realty/prodazha-kvartir/prodam-kvartiru-ordzhonikidzevskiy-23182085.html'),
 ('23182211',
  '/realty/prodazha-kvartir/prodam-kvartiru-ordzhonikidzevskiy-23182211.html'),
 ('23352148',
  '/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23352148.html'),
 ('23352147',
  '/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23352147.html'),
 ('23343164',
  '/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23343164.html'),
 ('23343161', '/realty/prodazha-kvartir/prodam-kvartiru-23343161.html'),
 ('23343160', '/realty/prodazha-kvartir/prodam-kvartiru-23343160.html'),
 ('23343159', '/realty/prodazha-kvartir/prodam-kvartiru-23343159.html'),
 ('23343158', '/realty/prodazha-kvartir/prodam-kvartiru-23343158.html'),
 ('23321318', '/realty/prodazha-kvartir/prodam-kvartiru-23321318.html'),
 ('23321317',
  '

In [11]:
#  Вынесем получение списка айдишек в отдельную функцию
SITE = "http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/"

def get_id_list(num_page):
    if num_page == 1:
        page = requests.get(SITE)
    else:
        page = requests.get(SITE + f'?p={num_page}')
    page = BeautifulSoup(page.text, 'lxml')
    tr_list = [td for td in page.find('table', 'table').find_all('tr', recursive=False) if td.find('td', 'notebook-column')]
    id_list = [i.find('input', {'name': 'cs_note_tg'})['value'] for i in tr_list]
    link_list = [i.find_all('a')[0]['href'] for i in tr_list]

    return id_list, link_list

In [12]:
full_id_list, full_link_list = [], []
for num_page in list_pages:
    id_list, link_list = get_id_list(num_page) 
    full_id_list.extend(id_list)
    full_link_list.extend(link_list)

full_id_list, len(full_id_list), full_link_list, len(full_link_list)

(['23177987',
  '23177988',
  '23182085',
  '23182211',
  '23352148',
  '23352147',
  '23343164',
  '23343161',
  '23343160',
  '23343159',
  '23343158',
  '23321318',
  '23321317',
  '23321316',
  '23164004',
  '22166819',
  '23311683',
  '23303021',
  '23282259',
  '23282258',
  '23282257',
  '23257068',
  '23257065',
  '23257064',
  '23257060',
  '23257059',
  '23257058',
  '23257055',
  '23257053',
  '23257052',
  '23257051',
  '23257050',
  '23257049',
  '23257047',
  '23257046',
  '23257044',
  '23257043',
  '23257041',
  '23257040',
  '23257038',
  '23257037',
  '23257036',
  '23257034',
  '23257033',
  '23257032',
  '23257031',
  '23257030',
  '23257029',
  '23257028',
  '23257026',
  '23257025',
  '23257020',
  '23257019',
  '23257018',
  '23257016',
  '23257015',
  '23257014',
  '23257013',
  '23257012',
  '23257011',
  '23257010',
  '23257009',
  '23257008',
  '23257007',
  '23257006',
  '23257005',
  '23257004',
  '23257003',
  '23257002',
  '23257000',
  '23256999',
  '232

In [13]:
len(set(full_id_list)), len(set(full_link_list))

(610, 610)

#### 2) Создаем колонки

Данные будут собираться по 11 параметрам: Ссылка, Цена, Район, Адрес, Этаж, Высота потолка, Планировка, Площадь, Состояние квартиры, Кол-во комнат, Описание.

Из объявлений квартир видно, что данные где-то заполнены не полностью, соответственно в конечной таблице у нас будут пропуски. Так как на каждой странице квартиры таблица с данными разного размера, то ориентироваться будем по названию строк, а затем вставлять их в нужную колонку в нашем датафрейме.

<img src='../datasets/flat_1.png' width=400> <img src='../datasets/flat_2.png' width=400>

In [14]:
# Создаем колонки
columns = ['Ссылка', 'Цена', 'Район', 'Адрес', 'Этаж',
           'Высота потолка', 'Планировка', 'Площадь', 
           'Состояние квартиры', 'Кол-во комнат', 'Описание']
columns

['Ссылка',
 'Цена',
 'Район',
 'Адрес',
 'Этаж',
 'Высота потолка',
 'Планировка',
 'Площадь',
 'Состояние квартиры',
 'Кол-во комнат',
 'Описание']

#### 3) Парсим данные со страницы и через словарь вставляем их в датафрем

Оказалось, что просто идентификатора объявления недостаточно, чтобы перейти на страницу этого объявления...

Так как:
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-23257029.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23257030.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-ordzhonikidzevskiy-23182211.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html

То есть если квартира в описании имеет название района, то в адресной строке добавляется название этого района, если же район не указан, то окончание строки просто "prodam-kvartiru-айди объявления". Следовательно, чтобы перейти на страницу объявления мы вынуждены вместе с айдишниками парсить район.

Тут я хотел было расстроиться, затем просто вписать вручную транслитерацию районов (их оказывается всего 4), но заметил, что все гораздо проще. Каждая строка таблицы (как же это было очевидно...) имеет внутри себя ссылку на само объявление. Поэтому вместе с айдишками, можно сразу собирать ссылки, которые потом использовать для перехода и сбора данных.

In [15]:
# Возвращаемся на пункт "Парсим идентификаторы квартир"
...
# Теперь все отлично, можно создать датафрейм

In [16]:
df = pd.DataFrame(index=full_id_list, columns=columns)
df['Ссылка'] = full_link_list
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23177987,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23177988,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23182085,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23182211,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23352148,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html
flat_page =  requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html')
flat_page = BeautifulSoup(flat_page.text, 'lxml')
flat_page

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="780e99506e03edb5" name="yandex-verification"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Квартира 61,30 м2 по улице Суворова 139 в Магнитогорске - продается за 4050000 рублей. Объявление от продавца  на CityStar.</title>
<meta content="Квартира" http-equiv="keywords"/>
<meta content="Продается Трехкомнатная квартира по адресу: Магнитогорск, ул. Суворова, 139,  район Правобережный за 4050000 рублей." http-equiv="description"/>
<link href="/img/favicon2.png" rel="SHORTCUT ICON"/>
<link href="http://r.citystar.ru/img/_min/realty.adv-card.min.css" rel="stylesheet" type="text/css"/>
<script src="http://r.citystar.ru/js/jquery-1.7.2.min.js?abcd2ba13348f178b17141b445bc99f1917d47af" type="text/javascript"></script>
<script src="http://r.citystar.ru/js/

In [18]:
div = flat_page.find('div', 'adv-main-data')
div

<div class="adv-main-data">
<table>
<tr class="gr-t1">
<td class="group-title" colspan="2">Трехкомнатная квартира по ул. Суворова, 139 в Магнитогорске</td>
</tr>
<tr>
<td class="field-title">Цена</td>
<td class="field"><span class="price">4 050 000</span> р. <span class="colored-text-1">(<strong>66 069</strong> р./м2)</span><br/><a class="common-button_1 card-hypotec-button" href="/mortgageBid/23188503">Подать заявку на ипотеку</a></td>
</tr>
<tr class="even">
<td class="field-title">Район</td>
<td class="field">Правобережный</td>
</tr>
<tr>
<td class="field-title">Адрес</td>
<td class="field">Суворова, 139</td>
</tr>
<tr class="even">
<td class="field-title">Этаж</td>
<td class="field">2<span class="slash">/</span>5</td>
</tr>
<tr>
<td class="field-title">Высота потолка</td>
<td class="field">2,5 м.</td>
</tr>
<tr class="even">
<td class="field-title">Планировка</td>
<td class="field">хрущевка</td>
</tr>
<tr>
<td class="field-title">Площадь</td>
<td class="field"><span class="colored-

In [19]:
tr_list = div.find_all('tr')
tr_list

[<tr class="gr-t1">
 <td class="group-title" colspan="2">Трехкомнатная квартира по ул. Суворова, 139 в Магнитогорске</td>
 </tr>,
 <tr>
 <td class="field-title">Цена</td>
 <td class="field"><span class="price">4 050 000</span> р. <span class="colored-text-1">(<strong>66 069</strong> р./м2)</span><br/><a class="common-button_1 card-hypotec-button" href="/mortgageBid/23188503">Подать заявку на ипотеку</a></td>
 </tr>,
 <tr class="even">
 <td class="field-title">Район</td>
 <td class="field">Правобережный</td>
 </tr>,
 <tr>
 <td class="field-title">Адрес</td>
 <td class="field">Суворова, 139</td>
 </tr>,
 <tr class="even">
 <td class="field-title">Этаж</td>
 <td class="field">2<span class="slash">/</span>5</td>
 </tr>,
 <tr>
 <td class="field-title">Высота потолка</td>
 <td class="field">2,5 м.</td>
 </tr>,
 <tr class="even">
 <td class="field-title">Планировка</td>
 <td class="field">хрущевка</td>
 </tr>,
 <tr>
 <td class="field-title">Площадь</td>
 <td class="field"><span class="colored

In [20]:
# field-title – ключ к словарю, field – его значение, поиск в словаре происходит за O(1), так что все ок
# Подумал сначала сразу получить разделенные данные, но получить их в грязном виде,
# а потом почистить пандасом оказалось проще, чем сразу собрать чистые как надо
# плюс это позволит не заморачиваться над тем, на какую позицию придется ставить найденный элемент

key_list = [key.get_text() for key in div.find_all('td', 'field-title')]
value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]

data_dict = dict(zip(key_list, value_list))
data_dict['Описание'] = div.find('td', 'note').get_text()
data_dict

# P.S. не понимал почему у одних строк на этой странице класс even, а другие без класса, а потом как дошло...

{'Цена': '4050000р. (66069р./м2)Подать заявку на ипотеку',
 'Район': 'Правобережный',
 'Адрес': 'Суворова, 139',
 'Этаж': '2/5',
 'Высота потолка': '2,5 м.',
 'Планировка': 'хрущевка',
 'Площадь': 'общая 61,3 м2, жилая 46 м2, кухни 6 м2',
 'Состояние квартиры': 'Среднее',
 'Кол-во комнат': 'Трехкомнатная',
 'Этажность дома': '5',
 'Описание': '🏠 Продается уютная 3-комнатная квартира 61,3 м² в отличном состоянии\n📍 Адрес: ул. Суворова, 139, 2 этаж 5-этажного блочного дома\n🔍 О квартире:\n•📐 Площадь: 61.3 м²\n•🏢 Этаж: 2/5\n•🧭 Ориентация окон: Восток-Запад → светлая, весь день солнце\n•🪟 Окна: Пластиковые стеклопакеты\n•🧊 Балкон: Застекленный с выносом\n•⚡ Электрика: Медная проводка\n•💧 Коммуникации: Трубы пластик, установлены водомеры\n•🚻 Санузел: Раздельный в кафеле\n•🧱 Отделка: Полы — стяжка, в коридоре — кафель. Стены ровные. Потолки: натяжной и двухуровневый из гипсокартона.\n🛋️ Остается вся мебель и техника:\n•🍽️ Кухня: кухонный гарнитур, обеденный стол\n•🚪 Прихожая: мебельная стенк

In [21]:
BASE_SITE = 'http://magnitogorsk-citystar.ru'

def get_data(link):
    flat_page = BeautifulSoup(requests.get('http://magnitogorsk-citystar.ru' + link).text, 'lxml')
    div = flat_page.find('div', 'adv-main-data')
    key_list = [key.get_text() for key in div.find_all('td', 'field-title')]
    value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]
    data_dict = dict(zip(key_list, value_list))
    if div.find('td', 'note'):
        data_dict['Описание'] = div.find('td', 'note').get_text()
    return data_dict

In [22]:
for ind in full_id_list:
    df.loc[str(ind), 'Цена':] = get_data(df.loc[str(ind), 'Ссылка'])

df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23177987,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000р. (83607р./м2)Подать заявку на ипотеку,Ленинский,"Октябрьская, 32/1",5/5,"2,5 м.",брежневка,"общая 30,5 м2, жилая 18 м2, кухни 6 м2",Среднее,Однокомнатная,Продаётся уютная однокомнатная квартира площад...
23177988,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000р. (68548р./м2)Подать заявку на ипотеку,Ленинский,"Ленина пр-т, 9",5/5,"3,2 м.",старой планировки,"общая 37,2 м2, жилая 24 м2, кухни 10 м2",Требуется ремонт,Однокомнатная,Продаётся уютная однокомнатная квартира площад...
23182085,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,4400000р. (67692р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Ворошилова, 35/1",7/9,"2,5 м.",улучшенная,"общая 65 м2, жилая 45 м2, кухни 9 м2",Хорошее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...
23182211,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,3950000р. (66949р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Тевосяна, 11/3",7/9,"2,5 м.",полусмежная,"общая 59 м2, жилая 41 м2, кухни 9 м2",Среднее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...
23352148,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,5000000р. (94340р./м2)Подать заявку на ипотеку,Ленинский,"Металлургов, 12",3/5,3 м.,старой планировки,"общая 53 м2, жилая 35,3 м2, кухни 9 м2",Идеальное,Двухкомнатная,Продаётся уютная двухкомнатная квартира площад...


#### 4) Приводим собранные данные в порядок

In [25]:
flat = df.loc['23188503']
flat

Ссылка                /realty/prodazha-kvartir/prodam-kvartiru-pravo...
Цена                     4050000р. (66069р./м2)Подать заявку на ипотеку
Район                                                     Правобережный
Адрес                                                     Суворова, 139
Этаж                                                                2/5
Высота потолка                                                   2,5 м.
Планировка                                                     хрущевка
Площадь                          общая 61,3 м2, жилая 46 м2, кухни 6 м2
Состояние квартиры                                              Среднее
Кол-во комнат                                             Трехкомнатная
Описание              🏠 Продается уютная 3-комнатная квартира 61,3 м...
Общая площадь                                                      61,3
Жилая площадь                                                        46
Площадь кухни                                                   

In [27]:
df[['Общая площадь', 'Жилая площадь', 'Площадь кухни']] = df['Площадь'].str.split(' ', expand=True).loc[:, [1, 4, 7]]
df.drop('Площадь', axis=1)
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни
23177987,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000р. (83607р./м2)Подать заявку на ипотеку,Ленинский,"Октябрьская, 32/1",5/5,"2,5 м.",брежневка,"общая 30,5 м2, жилая 18 м2, кухни 6 м2",Среднее,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"30,5",18,6
23177988,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000р. (68548р./м2)Подать заявку на ипотеку,Ленинский,"Ленина пр-т, 9",5/5,"3,2 м.",старой планировки,"общая 37,2 м2, жилая 24 м2, кухни 10 м2",Требуется ремонт,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"37,2",24,10
23182085,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,4400000р. (67692р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Ворошилова, 35/1",7/9,"2,5 м.",улучшенная,"общая 65 м2, жилая 45 м2, кухни 9 м2",Хорошее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,65,45,9
23182211,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,3950000р. (66949р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Тевосяна, 11/3",7/9,"2,5 м.",полусмежная,"общая 59 м2, жилая 41 м2, кухни 9 м2",Среднее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,59,41,9
23352148,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,5000000р. (94340р./м2)Подать заявку на ипотеку,Ленинский,"Металлургов, 12",3/5,3 м.,старой планировки,"общая 53 м2, жилая 35,3 м2, кухни 9 м2",Идеальное,Двухкомнатная,Продаётся уютная двухкомнатная квартира площад...,53,"35,3",9


In [28]:
df[['Этаж', 'Этажность дома']] = df['Этаж'].str.split('/', expand=True)
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
23177987,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000р. (83607р./м2)Подать заявку на ипотеку,Ленинский,"Октябрьская, 32/1",5,"2,5 м.",брежневка,"общая 30,5 м2, жилая 18 м2, кухни 6 м2",Среднее,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"30,5",18,6,5
23177988,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000р. (68548р./м2)Подать заявку на ипотеку,Ленинский,"Ленина пр-т, 9",5,"3,2 м.",старой планировки,"общая 37,2 м2, жилая 24 м2, кухни 10 м2",Требуется ремонт,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"37,2",24,10,5
23182085,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,4400000р. (67692р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Ворошилова, 35/1",7,"2,5 м.",улучшенная,"общая 65 м2, жилая 45 м2, кухни 9 м2",Хорошее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,65,45,9,9
23182211,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,3950000р. (66949р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Тевосяна, 11/3",7,"2,5 м.",полусмежная,"общая 59 м2, жилая 41 м2, кухни 9 м2",Среднее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,59,41,9,9
23352148,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,5000000р. (94340р./м2)Подать заявку на ипотеку,Ленинский,"Металлургов, 12",3,3 м.,старой планировки,"общая 53 м2, жилая 35,3 м2, кухни 9 м2",Идеальное,Двухкомнатная,Продаётся уютная двухкомнатная квартира площад...,53,"35,3",9,5


In [29]:
df['Цена'] = df['Цена'][:].str.split('р.', expand=True).loc[:, 0]
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
23177987,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000,Ленинский,"Октябрьская, 32/1",5,"2,5 м.",брежневка,"общая 30,5 м2, жилая 18 м2, кухни 6 м2",Среднее,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"30,5",18,6,5
23177988,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000,Ленинский,"Ленина пр-т, 9",5,"3,2 м.",старой планировки,"общая 37,2 м2, жилая 24 м2, кухни 10 м2",Требуется ремонт,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"37,2",24,10,5
23182085,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,4400000,Орджоникидзевский,"Ворошилова, 35/1",7,"2,5 м.",улучшенная,"общая 65 м2, жилая 45 м2, кухни 9 м2",Хорошее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,65,45,9,9
23182211,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,3950000,Орджоникидзевский,"Тевосяна, 11/3",7,"2,5 м.",полусмежная,"общая 59 м2, жилая 41 м2, кухни 9 м2",Среднее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,59,41,9,9
23352148,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,5000000,Ленинский,"Металлургов, 12",3,3 м.,старой планировки,"общая 53 м2, жилая 35,3 м2, кухни 9 м2",Идеальное,Двухкомнатная,Продаётся уютная двухкомнатная квартира площад...,53,"35,3",9,5


In [30]:
df['Описание'] = df['Описание'].str.replace('^id:\\d+\\.\\s*', '', regex=True)
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
23177987,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000,Ленинский,"Октябрьская, 32/1",5,"2,5 м.",брежневка,"общая 30,5 м2, жилая 18 м2, кухни 6 м2",Среднее,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"30,5",18,6,5
23177988,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,2550000,Ленинский,"Ленина пр-т, 9",5,"3,2 м.",старой планировки,"общая 37,2 м2, жилая 24 м2, кухни 10 м2",Требуется ремонт,Однокомнатная,Продаётся уютная однокомнатная квартира площад...,"37,2",24,10,5
23182085,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,4400000,Орджоникидзевский,"Ворошилова, 35/1",7,"2,5 м.",улучшенная,"общая 65 м2, жилая 45 м2, кухни 9 м2",Хорошее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,65,45,9,9
23182211,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,3950000,Орджоникидзевский,"Тевосяна, 11/3",7,"2,5 м.",полусмежная,"общая 59 м2, жилая 41 м2, кухни 9 м2",Среднее,Трехкомнатная,Продаётся уютная трёхкомнатная квартира площад...,59,41,9,9
23352148,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,5000000,Ленинский,"Металлургов, 12",3,3 м.,старой планировки,"общая 53 м2, жилая 35,3 м2, кухни 9 м2",Идеальное,Двухкомнатная,Продаётся уютная двухкомнатная квартира площад...,53,"35,3",9,5


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 610 entries, 23177987 to 14046477
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Ссылка              610 non-null    object
 1   Цена                609 non-null    object
 2   Район               197 non-null    object
 3   Адрес               603 non-null    object
 4   Этаж                610 non-null    object
 5   Высота потолка      30 non-null     object
 6   Планировка          62 non-null     object
 7   Площадь             610 non-null    object
 8   Состояние квартиры  68 non-null     object
 9   Кол-во комнат       597 non-null    object
 10  Описание            608 non-null    object
 11  Общая площадь       610 non-null    object
 12  Жилая площадь       610 non-null    object
 13  Площадь кухни       610 non-null    object
 14  Этажность дома      610 non-null    object
dtypes: object(15)
memory usage: 92.4+ KB


#### 5) Сохраням файл в удобный формат

In [31]:
df.to_csv('../datasets/flat_magnitogorsk.csv')

### Заключение

Сайт делал какой-то извращенец, задание интересное, понравилось, ≈6 часов точно ушло. 
- Парсер первый — айди и ссылки на страницы квартир работает около 2 минут.
- Парсер второй – основные данные по квартирам работает около 5 минут.

Осталось только преобразовать в необходимые типы данных.

Доброй ночи 🌛